In [1]:
# making glove vectors for embedding layer
#  https://www.dlology.com/blog/simple-stock-sentiment-analysis-with-news-data-in-keras/

In [1]:
import os
import pickle 
import json
import numpy as np
FilePAth=r''
FilePath=''
import pandas as pd
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU,LSTM
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler

In [3]:
embeddings = {}
with open(FilePAth+'\glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        
      
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
#         print(word)
#         print(coeffs)
        embeddings[word] = coeffs

In [4]:
embeddings['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [5]:
# with open(FilePAth+'embeddings_dict.txt','w') as f:
#   f.write(str(embeddings))

In [6]:
# embeddings=None
# with open(FilePAth+'embeddings_dict.txt','r') as f:
#   train_descriptions=f.read()
# json_sting=embeddings.replace("'","\"")
# embeddings=json.loads(json_sting)

In [7]:
X_test=[]
with open(FilePath+'testheadlines.pkl', 'rb') as f:
    X_test=pickle.load(f)

In [8]:
X_train=[]
with open(FilePath+'trainheadlines.pkl', 'rb') as f:
    X_train=pickle.load(f)

In [9]:
len(X_train)

1591

In [10]:
y_train=[]
with open(FilePath+'y_train.pkl', 'rb') as f:
    y_train=pickle.load(f)

In [11]:
y_test=[]
with open(FilePath+'y_test.pkl', 'rb') as f:
    y_test=pickle.load(f)

Now deciding the number of max len to be choosen

In [12]:
X_train_lengths = list(map(lambda x: len(x.split()), X_train))
df = pd.DataFrame({'counts': X_train_lengths})
df.counts.plot.hist(bins = 40)

HEnce most of the string have length of 500 and maximum of 700 so padding string to 500

In [13]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [14]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(FilePAth+'\glove.6B.50d.txt')

In [15]:
word_to_vec_map['cat'].shape


(50,)

In [16]:
def sentences_to_indices(X, word_to_index, max_len):
  
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape
    X_indices = np.zeros((m, max_len), dtype=int)
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j += 1
                if j >= max_len:
                    break
            
    return X_indices

In [17]:
X1 = np.array(['germany to meet with france'])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 9)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['germany to meet with france']
X1_indices = [[160180 360915 239785 388711 152927      0      0      0      0]]


In [18]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len=len(word_to_index)+1
    emb_dim = word_to_vec_map["cucumber"].shape[0]   #50
    
    # now create embeddings for all word in X
    
    emb_matrix =np.zeros((vocab_len,emb_dim))
    
    for word,index in word_to_index.items():
        emb_matrix[index,:]=word_to_vec_map[word]
        
    # now create embedding layer which is alredy trained to emd_matrix
    # so set embedding layer trainable =false
    
    embedding_layer =Embedding(vocab_len,emb_dim,trainable=False)
    
    embedding_layer.build((None,))
    
    # set weights of embedding layer or embedding matrix
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer
    
    
    
    

In [19]:

embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embedding_layer.get_weights()[0][word_to_index['cat']]

array([ 0.45281 , -0.50108 , -0.53714 , -0.015697,  0.22191 ,  0.54602 ,
       -0.67301 , -0.6891  ,  0.63493 , -0.19726 ,  0.33685 ,  0.7735  ,
        0.90094 ,  0.38488 ,  0.38367 ,  0.2657  , -0.08057 ,  0.61089 ,
       -1.2894  , -0.22313 , -0.61578 ,  0.21697 ,  0.35614 ,  0.44499 ,
        0.60885 , -1.1633  , -1.1579  ,  0.36118 ,  0.10466 , -0.78325 ,
        1.4352  ,  0.18629 , -0.26112 ,  0.83275 , -0.23123 ,  0.32481 ,
        0.14485 , -0.44552 ,  0.33497 , -0.95946 , -0.097479,  0.48138 ,
       -0.43352 ,  0.69455 ,  0.91043 , -0.28173 ,  0.41637 , -1.2609  ,
        0.71278 ,  0.23782 ], dtype=float32)

In [20]:
print('Build model...')
model = Sequential()
model.add(pretrained_embedding_layer(word_to_vec_map, word_to_index))
model.add(LSTM(128, dropout=0.3, return_sequences=True)) 
model.add(LSTM(128, dropout=0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [21]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 20,223,411
Trainable params: 223,361
Non-trainable params: 20,000,050
_________________________________________________________________


In [22]:

maxlen = 500
batch_size = 32
X_train_indices = sentences_to_indices(X_train, word_to_index, maxlen)
X_test_indices = sentences_to_indices(X_test, word_to_index, maxlen)

In [23]:
Y_train = y_train.reshape((-1,1))
Y_test = y_test.reshape((-1,1))


In [24]:
print('Train...')
history = model.fit(X_train_indices, Y_train, batch_size=batch_size, epochs=10,
          validation_data=(X_test_indices, Y_test))

model.save("./model.h5")
score, acc = model.evaluate(X_test_indices, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)






Train...
Train on 1591 samples, validate on 398 samples
Epoch 1/10
1591/1591 [==============================] - 104s 65ms/step - loss: 0.6956 - accuracy: 0.5261 - val_loss: 0.6941 - val_accuracy: 0.5050
Epoch 2/10
1591/1591 [==============================] - 92s 58ms/step - loss: 0.6910 - accuracy: 0.5399 - val_loss: 0.6971 - val_accuracy: 0.5050
Epoch 3/10
1591/1591 [==============================] - 100s 63ms/step - loss: 0.6880 - accuracy: 0.5475 - val_loss: 0.6954 - val_accuracy: 0.5075
Epoch 4/10
1591/1591 [==============================] - 99s 62ms/step - loss: 0.6859 - accuracy: 0.5531 - val_loss: 0.7021 - val_accuracy: 0.5126
Epoch 5/10
1591/1591 [==============================] - 99s 62ms/step - loss: 0.6876 - accuracy: 0.5512 - val_loss: 0.6949 - val_accuracy: 0.5151
Epoch 6/10
1591/1591 [==============================] - 100s 63ms/step - loss: 0.6834 - accuracy: 0.5651 - val_loss: 0.6957 - val_accuracy: 0.5101
Epoch 7/10
1591/1591 [==============================] - 118s 74ms

In [25]:
# X_train